In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
from tqdm import tqdm

In [ ]:
# ------------------------------
# Parameters
# ------------------------------
ETFS             = ["XLY","XLP","XLE","XLF","XLV","XLI","XLB","XLK","XLU"]
START            = "2015-01-01"
LOOKBACK_MONTHS  = 6
TOP_N            = 3
TC               = 0.001  # transaction cost per round-trip

In [ ]:
# ------------------------------
# Download & prepare data
# ------------------------------
prices        = yf.download(ETFS + ["SPY"], start=START, auto_adjust=True)["Close"]
returns       = prices.pct_change().fillna(0)
price_monthly = prices[ETFS].resample("ME").last()

In [ ]:
# ------------------------------
# Clean Sector Rotation vs. SPY
# ------------------------------
momentum             = price_monthly.divide(price_monthly.shift(LOOKBACK_MONTHS)).subtract(1)
rank_pct             = momentum.rank(pct=True, axis=1)
signal               = (rank_pct >= (1 - TOP_N/len(ETFS))).astype(float)
positions            = signal.reindex(prices.index, method="ffill")
positions            = positions.div(positions.sum(axis=1), axis=0).fillna(0)
strategy_returns     = (positions.shift() * returns[ETFS]).sum(axis=1)
strategy_returns    -= positions.diff().abs().sum(axis=1) * TC
cumulative_strategy  = (1 + strategy_returns).cumprod()
spy_returns          = returns["SPY"]
cumulative_spy       = (1 + spy_returns).cumprod().reindex(cumulative_strategy.index, method="ffill")

plt.figure(figsize=(10,5))
plt.plot(cumulative_strategy, label="6M Sector Rotation")
plt.plot(cumulative_spy, label="SPY Buy-Hold")
plt.title("6-Month Momentum Sector Rotation vs. SPY")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ------------------------------
# Performance metrics
# ------------------------------
def perf_stats(rts, cum_rts):
    ann      = rts.mean() * 252
    vol      = rts.std(ddof=0) * np.sqrt(252)
    sharpe   = ann/vol if vol != 0 else np.nan
    drawdown = (cum_rts.cummax() - cum_rts).max()
    return ann, vol, sharpe, drawdown

s_ann, s_vol, s_sharpe, s_dd = perf_stats(strategy_returns, cumulative_strategy)
b_ann, b_vol, b_sharpe, b_dd = perf_stats(
    spy_returns.loc[cumulative_strategy.index], cumulative_spy
)
print(f"{'Metric':20}{'Strategy':>12}{'SPY':>12}")
print("-" * 44)
print(f"{'Ann. Return':20}{s_ann:12.2%}{b_ann:12.2%}")
print(f"{'Ann. Volatility':20}{s_vol:12.2%}{b_vol:12.2%}")
print(f"{'Sharpe Ratio':20}{s_sharpe:12.2f}{b_sharpe:12.2f}")
print(f"{'Max Drawdown':20}{s_dd:12.2%}{b_dd:12.2%}")

In [ ]:
# ------------------------------
# Regime-Adaptive Sector Rotation
# ------------------------------
ma200       = prices['SPY'].rolling(200).mean()
bull_regime = (prices['SPY'] > ma200).astype(float)
pos_reg     = positions.multiply(bull_regime, axis=0)
pos_reg['SPY'] = 1 - bull_regime

def backtest(pos_df, ret_df):
    strat = (pos_df.shift() * ret_df).sum(axis=1)
    tc    = pos_df[ETFS].diff().abs().sum(axis=1) * TC
    return (strat - tc).fillna(0)

reg_returns = backtest(pos_reg, returns)
cum_reg     = (1 + reg_returns).cumprod()

plt.figure(figsize=(10,5))
plt.plot(cumulative_strategy, label='Always Rotation')
plt.plot(cum_reg, label='Regime-Adaptive Rotation')
plt.plot(cumulative_spy, label='SPY Buy-Hold')
plt.fill_between(
    cum_reg.index, 0, 1,
    where=(bull_regime == 0), color='gray', alpha=0.2,
    transform=plt.gca().get_xaxis_transform()
)
plt.title('Regime-Adaptive Sector Rotation vs. SPY')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# ------------------------------
# Performance Attribution
# ------------------------------
daily_contrib = positions.shift() * returns[ETFS]
total_contrib = daily_contrib.sum()
avg_weights   = positions.mean()
attr = pd.DataFrame({
    'Total Return': total_contrib,
    'Avg Weight'  : avg_weights
}).sort_values('Total Return', ascending=False)

print('=== Sector Attribution ===')
print(attr.to_string(float_format='{:.2%}'.format))

fig, ax1 = plt.subplots(figsize=(8,4))
attr['Total Return'].plot.bar(ax=ax1, rot=45, width=0.4, label='Total Return')
ax2 = ax1.twinx()
attr['Avg Weight'].plot.bar(ax=ax2, rot=45, width=0.4, label='Avg Weight')
ax1.set_ylabel('Total Return')
ax2.set_ylabel('Avg Weight')
ax1.set_title('Sector Attribution')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.tight_layout()
plt.show()

In [ ]:
# ------------------------------
# Parameter Sweep & Heatmap
# ------------------------------
lookbacks = [3, 6, 9, 12]
top_ns    = [2, 3, 4, 5]
results   = []
for lb in lookbacks:
    mom_lb = price_monthly.divide(price_monthly.shift(lb)).subtract(1)
    pct_lb = mom_lb.rank(pct=True, axis=1)
    for tn in top_ns:
        sig = (pct_lb >= (1 - tn/len(ETFS))).astype(float)
        pos = sig.reindex(prices.index, method='ffill')
        pos = pos.div(pos.sum(axis=1), axis=0).fillna(0)
        ret = (pos.shift() * returns[ETFS]).sum(axis=1)
        ret -= pos.diff().abs().sum(axis=1) * TC
        cum = (1 + ret).cumprod()
        ann    = ret.mean()*252
        vol    = ret.std(ddof=0)*np.sqrt(252)
        sharpe = ann/vol if vol!=0 else np.nan
        results.append({'lookback': lb, 'top_n': tn, 'Sharpe': sharpe})
df_res = pd.DataFrame(results)
print('Sweep shape:', df_res.shape)
print(df_res.head(8))
df_pivot = df_res.pivot_table(
    index='lookback', columns='top_n', values='Sharpe', aggfunc='first'
)
print(df_pivot)

In [ ]:
# Plot Sharpe Ratio Heatmap
fig, ax = plt.subplots(figsize=(6,4))
data = df_pivot.values
im   = ax.imshow(data, origin='lower', aspect='auto', cmap='viridis')
ax.set_xticks(np.arange(data.shape[1]))
ax.set_yticks(np.arange(data.shape[0]))
ax.set_xticklabels(df_pivot.columns)
ax.set_yticklabels(df_pivot.index)
plt.setp(ax.get_xticklabels(), rotation=45, ha='right')
cbar = fig.colorbar(im, ax=ax, label='Sharpe Ratio')
for i in range(data.shape[0]):
    for j in range(data.shape[1]):
        ax.text(j, i, f"{data[i,j]:.2f}",
                ha='center', va='center',
                color='white' if data[i,j] < data.max()/2 else 'black')
ax.set_xlabel('Top N ETFs')
ax.set_ylabel('Lookback (months)')
ax.set_title('Sharpe Ratio Heatmap')
plt.tight_layout()
plt.show()